In [ ]:
import pandas as pd
import syft as sy
import numpy as np
# sy.logger.remove()

### Logging into the domain Nodes

In [ ]:
# We will login into Canada and Italy domain node
ca = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
it = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)

In [ ]:
# ca.store

In [ ]:
# it.store

In [13]:
ca_data = ca.store['data1']
it_data = it.store['data1']

In [9]:
ca_data.shape

<TensorPointer -> canada:bdf0b7a3b34c47c1895436334a63a17b>

In [12]:
# result = ca_data + it_data

In [ ]:
self = ca_data
other = it_data

In [ ]:
# Conveca_data ca_dataset_ptr to mpc tensor

parties = [self.client, other.client]

self_mpc = MPCTensor(secret=self, shape=(40000, 3), parties=parties)
other_mpc = MPCTensor(secret=other, shape=(40000, 3), parties=parties)

In [11]:
result.request("asdfasdfasdfasfasfasfasfasdf")

In [11]:
result.get()

array([[      2,     526,   11184],
       [      2,    1178,  251483],
       [      2,    1254, 1809585],
       ...,
       [      3,    1218,     295],
       [      3,     673,    3012],
       [      3,     861,    2887]])

In [12]:
from syft.core.tensor.smpc.mpc_tensor import MPCTensor

[2021-08-19T19:27:30.111696+0100][CRITICAL][logger]][64569] __getattribute__ failed. If you are trying to access an EnumAttribute or a StaticAttribute, be sure they have been added to the AST. Falling back on__getattr__ to search in self.attrs for the requested field.
[2021-08-19T19:27:30.113661+0100][CRITICAL][logger]][64569] 'Class' object has no attribute 'generate_przs'
[2021-08-19T19:27:30.332322+0100][CRITICAL][logger]][64569] __getattribute__ failed. If you are trying to access an EnumAttribute or a StaticAttribute, be sure they have been added to the AST. Falling back on__getattr__ to search in self.attrs for the requested field.
[2021-08-19T19:27:30.333720+0100][CRITICAL][logger]][64569] 'Class' object has no attribute 'generate_przs'
[2021-08-19T19:27:30.563990+0100][CRITICAL][logger]][64569] __getattribute__ failed. If you are trying to access an EnumAttribute or a StaticAttribute, be sure they have been added to the AST. Falling back on__getattr__ to search in self.attrs fo

In [9]:
result = ca_mpc + it_mpc

A
C
C.1
C.N.0
C.N.1
C.N.1.A
<built-in function add>
add
D
D.2
Shape seems to be:(40000, 3) of type <class 'tuple'>


In [10]:
result.request("asdfjas;ofajwfawef")

In [11]:
result.get()

array([[      2,     526,   11184],
       [      2,    1178,  251483],
       [      2,    1254, 1809585],
       ...,
       [      3,    1218,     295],
       [      3,     673,    3012],
       [      3,     861,    2887]])

In [13]:
import operator

In [11]:
ca_child = ca_mpc.child[0]
it_child = it_mpc.child[0]

In [22]:
self = ca_mpc
other = it_mpc


In [25]:
mpc_tensor.request("asdfasfadsfasfasdfasfasdfasdfasfasfasdfasfasdfasdfadsf")

In [15]:
result_ca_share.request("I REALLY WANT IT!!!")

In [16]:
result_ca_share.get()

ShareTensor(child=[[  120411545686243275  5420364468416716212  4924260048870791973]
 [-8641641401201989031 -2149982478569540580 -6212756238532366336]
 [ 5415280196075540049  8659212241275543408  2419037793033825306]
 ...
 [-5157422069723653582 -1496009065233679124 -4812975320743786707]
 [-7522203994420192663 -7042870547401880629 -2297386771529147739]
 [-5308235108074479184 -6198067076408611232  1545530758790299955]])

In [43]:
ca_child = ca_mpc.child[1]
it_child = it_mpc.child[1]

In [44]:
result_ca_share = ca_child + it_child

In [45]:
result_ca_share.request("I REALLY WANT IT!!!")

In [46]:
result_ca_share.get()

ShareTensor(child=[[ 3789326825264276718 -6195432187784671569  7724877658211289854]
 [ 5086638945853558380 -4163047212382163728  7873674670543014044]
 [ 8819932998979885411   566006346165101622 -1466510659657682348]
 ...
 [ -813453231418351047  1459449186582648287  1084914133896341587]
 [ 6387086361537578087 -6418132966490085138  9206396216059676139]
 [-1053403205987492338 -4350634154132594725  7797576070600199993]])

In [17]:
result = ca_mpc + it_mpc

/Users/atrask/Laboratory/openmined/PySyft/packages/syft/src/syft/core/tensor/smpc/mpc_tensor.py:228: RuntimeWarning: invalid value encountered in add
  res = op(np.empty(x_shape), np.empty(y_shape)).shape


In [19]:
result.request("Asdfasdfasfsdf")

In [61]:
for c in result.child:
    c.request("Asdfasfasfasdf")

In [63]:
c.client.name

'istat'

In [65]:
c.client

<DomainClient: <UID: 683015daa92e43989c58e361eb641679>>

In [17]:
# Performing a Private / Public Addition

# Creating an array of ones of size (40000, 3)
# Converting float to int, for SMPC to work
e = np.ones((40000, 3))
e = e.astype(int)

result = mpc_tensor + e

In [18]:
result.request("asdfasdfadsdfasdf")

In [20]:
result.get()

array([[      2,     491,    9286],
       [      2,     819,  116605],
       [      2,     827, 1495176],
       ...,
       [      2,     599,     281],
       [      2,     441,    2988],
       [      2,     157,    2864]])